# Data Cleaning

### 1. Setup and Data Import

In [1]:
import pandas as pd
import numpy as np

In [2]:
cal_df = pd.read_csv('data/raw/calendar2024.csv')
lis_df = pd.read_csv('data/raw/listings2024.csv') 
rev_df = pd.read_csv('data/raw/reviews2024.csv')

### 2. Initial Column Cleanup

In [3]:
# Remove empty columns
null_cols_lis = lis_df.columns[lis_df.isna().all()].tolist()
lis_df = lis_df.drop(columns=null_cols_lis)

# Drop unnecessary columns
lis_df.drop(columns=['scrape_id', 'host_name', 'picture_url', 'host_url', 'host_thumbnail_url', 'host_picture_url'], inplace=True)
cal_df.drop(columns=['adjusted_price'], inplace=True)
rev_df.drop(columns=['reviewer_name'], inplace=True)

### 3. Data Type Conversions

In [4]:
# Helper functions
def convert_to_boolean(df, columns, true_value='t'):
    """Convert specified columns from string indicators to boolean"""
    for col in columns:
        df[col] = df[col] == true_value
    return df

def convert_to_datetime(df, columns):
    """Convert specified columns to datetime"""
    for col in columns:
        df[col] = pd.to_datetime(df[col])
    return df

def convert_to_type(df, columns, dtype):
    """Convert specified columns to given dtype"""
    for col in columns:
        df[col] = df[col].astype(dtype)
    return df

### 4. Apply Type Conversions

In [5]:
# Boolean conversions
boolean_cols = ['instant_bookable', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability']
lis_df = convert_to_boolean(lis_df, boolean_cols)
cal_df['available'] = cal_df['available'] == 't'

# Datetime conversions
datetime_cols_lis = ['calendar_last_scraped', 'first_review', 'last_review', 'last_scraped', 'host_since']
lis_df = convert_to_datetime(lis_df, datetime_cols_lis)
cal_df['date'] = pd.to_datetime(cal_df['date'])
rev_df['date'] = pd.to_datetime(rev_df['date'])

# String conversions
string_columns = ['bathrooms_text', 'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type', 'host_location', 'host_about', 'host_neighbourhood', 'listing_url', 'host_response_time', 'source', 'name', 'description', 'neighborhood_overview']
lis_df = convert_to_type(lis_df, string_columns, "string")
rev_df['comments'] = rev_df['comments'].astype("string")

### 5. Handle Percentage and Currency Columns

In [6]:
# Convert percentage columns
percentage_cols = ['host_response_rate', 'host_acceptance_rate']
for col in percentage_cols:
    lis_df = lis_df.rename(columns={col: f"{col}_pct"})
    lis_df[f"{col}_pct"] = lis_df[f"{col}_pct"].str.rstrip('%').astype('float') / 100

# Clean and standardize price columns
#DKK
lis_df['price'] = lis_df['price'].str.replace(r'[\$,]', '', regex=True)
lis_df = lis_df.rename(columns={'price': 'price_DKK'})
lis_df['price_DKK'] = pd.to_numeric(lis_df['price_DKK'], errors='coerce')

# Drop the 4 most expensive listings (they looked like ingenuine outliers)
expensive_indices = lis_df['price_DKK'].nlargest(4).index
lis_df = lis_df.drop(expensive_indices)

#USD
cal_df['price'] = cal_df['price'].str.replace(r'[\$,]', '', regex=True)
cal_df = cal_df.rename(columns={'price': 'price_USD'})
cal_df['price_USD'] = pd.to_numeric(cal_df['price_USD'], errors='coerce')

### 6. Process array-Type Columns

In [7]:
# Process amenities
lis_df['amenities_count'] = lis_df.amenities.str.strip('[]').str.split(',').str.len()

def clean_amenity(text):
    """Clean individual amenity strings"""
    import re
    text = str(text) # Convert to string if not already
    text = text.strip().strip('"\'').strip('.- ') # Basic cleaning
    text = text.encode('ascii', 'ignore').decode('ascii') # Replace unicode escape sequences with their characters
    text = re.sub(r'\s+', ' ', text) # Replace multiple spaces with single space
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Remove special characters, keeping only alphanumeric and spaces
    text = text.lower().strip() # Convert to lowercase, strip again, and remove any remaining leading/trailing spaces
    return text

# Clean and convert amenities to comma-separated string
lis_df['amenities'] = lis_df['amenities'].str.strip('[]').str.split(',').apply(
    lambda x: ','.join(
        sorted(  # Sort for consistency
            filter(None,  # Remove empty strings
                [clean_amenity(item) for item in x]
            )
        )
    )
)

lis_df['amenities'] = lis_df['amenities'].astype('string') # Convert to string dtype




# Process host verifications
# Count number of verifications per host
lis_df['host_verifications_count'] = lis_df['host_verifications'].str.strip('[]').str.split(', ').str.len()
lis_df['host_verifications'] = lis_df['host_verifications'].str.strip('[]').str.replace("'", "").str.split(', ')
# Create one-hot encoded columns
verification_dummies = lis_df['host_verifications'].str.join('|').str.get_dummies()
verification_dummies = verification_dummies.add_prefix('verification_')
lis_df = pd.concat([lis_df, verification_dummies], axis=1)

lis_df.drop(columns=['host_verifications', 'amenities'], inplace=True)

In [8]:
lis_df.to_parquet('data/processed/02_listings.parquet')
cal_df.to_parquet('data/processed/02_calendar.parquet')
rev_df.to_parquet('data/processed/02_reviews.parquet')